# Exploring Data Using Pandas

This is the first of four sessions looking at how to explore data in
Python. This session will focus on introducing the Python library,
[pandas](https://pandas.pydata.org/docs/). We will use pandas to import,
inspect, summarise, and transform the data, illustrating a typical
exploratory data analysis workflow.

We are using [Australian weather
data](https://www.kaggle.com/datasets/jsphyg/weather-dataset-rattle-package),
taken from [Kaggle](https://kaggle.com). This dataset is used to build
machine learning models that predict whether it will rain tomorrow,
using data about the weather every day from 2007 to 2017. To download
the data, click <a href="data/weatherAUS.csv" download>here</a>.

> **Please note!**
>
> We assume that you’ve created a `uv` project with packages `jupyter`,
> `pandas`, `numpy` and `seaborn`, and have created a blank notebook in
> this project and opened it in VS Code. If you’re not sure how to do
> this, please refer back to sessions [1
> (onboarding)](../01-onboarding/index.qmd) and [2
> (jupyter)](../02-jupyter_notebooks/index.qmd).

## Prep work

Place the `weatherAUS.csv` file you downloaded in a folder called `data`
within your project folder.

There’s also one more package we need, called `skimpy`. In your terminal
(PowerShell), from within your project folder:

``` sh
uv add skimpy
```

Then, in your JuPyter notebook in VS Code, create a code cell and start
by importing the packages we’ll need.

In [1]:
# Importing packages. This won't work if we didn't add them to our project using uv earlier.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from skimpy import skim

The bits after `as` (`plt`, `np`, `pd`, etc.) are called aliases. These
are just for convenience and aren’t essential, but it’s easier to type
`plt` every time rather than `matplotlib.pyplot`, for example. We can
choose whatever alias we want but it’s good to stick with common
conventions like `pd` for pandas - this is especially useful for
colleagues who might need to read your code further down the line (and
who will likely understand what `pd` is without having to look back to
where you imported your packages).

Next, we’re going to import the data using `read_csv` from `pandas`.
We’re putting our data into a *dataframe* which we’re giving the name
`df` - again, we could have chosen whatever name we liked.

In [2]:
df = pd.read_csv('data/weatherAUS.csv')

## Setting the Scene

Before we start to explore any dataset, we need to establish what we are
looking to do with the data. This should inform our decisions with any
exploration, and any analysis that follows.

**Questions:**

-   What are we trying to achieve?
-   How do our goals impact our analysis?
-   What should we take into consideration before we write any code?
-   What sort of questions might we be interested in with this dataset?

### What Our Data Can Tell Us (And What it Can’t)

We also need to consider what the data is and where it came from.

**Questions:**

-   How was the data collected?
-   What is it missing?
-   What do the variables in our dataset actually mean, and are they a
    good approximation of the concepts we are interested in?

## Exploring the Dataset

First, we should start with dataset-wide operations.

**Questions:**

-   What do we want to know about a dataset when we first encounter it?
-   How do we get a quick overview of the data that can help us in our
    next steps?
-   We need to get a “feel” for the data before we can really make any
    decisions about how to analyse it. How do we get there with a new
    dataset?

We can start by getting a quick glance at the data. The starting point
when you have just imported a new dataset is usually the pandas method
`.head(n)`, which shows the top $n$ rows of the dataset (by default, if
we don’t specify a number, it shows the top five rows).

In [3]:
# view the top five rows
df.head()

5 rows × 23 columns

You can also look at the bottom rows of the dataset, using `.tail(n)`.
This might be useful if you are dealing with time-series data. Below, we
specify that we want to look at the bottom ten rows.

In [4]:
# view the bottom ten rows
df.tail(10)

10 rows × 23 columns

A quick glimpse at the data is useful, but we may also want to get quick
descriptions of several aspects of the data. Such as the length of the
dataset (`len()`, which can also be used to get the length of various
Python objects), which tells us how many observations we have.

In [5]:
# get the object length
len(df)

145460

Another option is `pd.DataFrame.shape()`, which shows the length (number
of rows) and width (number of columns).

In [6]:
# get the object shape (number of rows, number of columns)
df.shape

(145460, 23)

Speaking of columns, if we want a quick list of the column names, we can
get this using `pd.DataFrame.columns()`.

In [7]:
# get all column names
df.columns

Index(['Date', 'Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation',
       'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow'],
      dtype='object')

A quick and easy way to get some valuable information about the dataset
is `pd.DataFrame.info()`, including the total non-null observations and
data type[1] of each column.

[1] For more information about pandas data types, check out the pandas
documentation on
[dtypes](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#basics-dtypes).

In [8]:
# get dataframe info (column indices, non-null counts, data types)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

If we wanted to get a better sense of the null values in each column, we
could calculate the percentage of null values by capturing whether each
row of each column is null (`pd.DataFrame.isnull()`), summing the total
null values in each column (`pd.DataFrame.sum()`), and then dividing by
the length of the dataframe (`/len()`).

In [9]:
# calculate the percentage of null values in each column
df.isnull().sum()/len(df)

Date             0.000000
Location         0.000000
MinTemp          0.010209
MaxTemp          0.008669
Rainfall         0.022419
Evaporation      0.431665
Sunshine         0.480098
WindGustDir      0.070989
WindGustSpeed    0.070555
WindDir9am       0.072639
WindDir3pm       0.029066
WindSpeed9am     0.012148
WindSpeed3pm     0.021050
Humidity9am      0.018246
Humidity3pm      0.030984
Pressure9am      0.103568
Pressure3pm      0.103314
Cloud9am         0.384216
Cloud3pm         0.408071
Temp9am          0.012148
Temp3pm          0.024811
RainToday        0.022419
RainTomorrow     0.022460
dtype: float64

If we want a quick summary of all the numeric columns in the dataset, we
can use `pd.DataFrame.describe()`.

In [10]:
# quick summary of numeric variables
df.describe()

However, I prefer to bring in another package, skimpy, that does all of
this very quickly and cleanly. We can get a detailed description of the
entire dataset using `skim()`.

In [11]:
# a more informative summary function from the skimpy package
skim(df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│ Data Summary Data Types │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓ │
│ ┃ Dataframe ┃ Values ┃ ┃ Column Type ┃ Count ┃ │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩ │
│ │ Number of rows │ 145460 │ │ float64 │ 16 │ │
│ │ Number of columns │ 23 │ │ string │ 7 │ │
│ └───────────────────┴────────┘ └─────────────┴───────┘ │
│ number │
│ ┏━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━┳━━━━━━━━┓ │
│ ┃ column ┃ NA ┃ NA % ┃ mean ┃ sd ┃ p0 ┃ p25 ┃ p50 ┃ p75 ┃ p100 ┃ hist ┃ │
│ ┡━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━╇━━━━━━━━┩ │
│ │ MinTemp │ 1485 │ 1.0208992162793895 │ 12.19 │ 6.398 │ -8.5 │ 7.6 │ 12 │ 16.9 │ 33.9 │ ▃▇▇▃ │ │
│ │ MaxTemp │ 1261 │ 0.8669049910628351 │ 23.22 │ 7.119 │ -4.8 │ 17.9 │ 22.6 │ 28.2 │ 48.1 │ ▁▇▇▃ │ │
│ │ Rainfall │ 3261 │ 2.241853430496356 │ 2.361 │ 8.478 │ 0 │ 0 │ 0 │ 0.8 │ 371 │ ▇ │ │
│ │ Evaporation │ 62790 │ 43.1665062560154 │ 5.468 │ 4.194 │ 0 │ 2.6 │ 4.8 │ 7.4 │ 145 │ ▇ │ │
│ │ Sunshine │ 69835 │ 48.00976213391998 │ 7.611 │ 3.785 │ 0 │ 4.8 │ 8.4 │ 10.6 │ 14.5 │ ▃▃▅▆▇▃ │ │
│ │ WindGustSpeed │ 10263 │ 7.055547916953114 │ 40.04 │ 13.61 │ 6 │ 31 │ 39 │ 48 │ 135 │ ▂▇▂ │ │
│ │ WindSpeed9am │ 1767 │ 1.214766946239516 │ 14.04 │ 8.915 │ 0 │ 7 │ 13 │ 19 │ 130 │ ▇▂ │ │
│ │ WindSpeed3pm │ 3062 │ 2.105046060772721 │ 18.66 │ 8.81 │ 0 │ 13 │ 19 │ 24 │ 87 │ ▅▇▂ │ │
│ │ Humidity9am │ 2654 │ 1.8245565791282827 │ 68.88 │ 19.03 │ 0 │ 57 │ 70 │ 83 │ 100 │ ▁▂▇▇▆ │ │
│ │ Humidity3pm │ 4507 │ 3.09844630826344 │ 51.54 │ 20.8 │ 0 │ 37 │ 52 │ 66 │ 100 │ ▁▅▆▇▅▂ │ │
│ │ Pressure9am │ 15065 │ 10.356799120033 │ 1018 │ 7.107 │ 980.5 │ 1013 │ 1018 │ 1022 │ 1041 │ ▂▇▅ │ │
│ │ Pressure3pm │ 15028 │ 10.331362573903478 │ 1015 │ 7.037 │ 977.1 │ 1010 │ 1015 │ 1020 │ 1040 │ ▂▇▅ │ │
│ │ Cloud9am │ 55888 │ 38.42155919153032 │ 4.447 │ 2.887 │ 0 │ 1 │ 5 │ 7 │ 9 │ ▇▂▃▂▇▅ │ │
│ │ Cloud3pm │ 59358 │ 40.80709473394748 │ 4.51 │ 2.72 │ 0 │ 2 │ 5 │ 7 │ 9 │ ▆▂▃▂▇▃ │ │
│ │ Temp9am │ 1767 │ 1.214766946239516 │ 16.99 │ 6.489 │ -7.2 │ 12.3 │ 16.7 │ 21.6 │ 40.2 │ ▂▇▇▃ │ │
│ │ Temp3pm │ 3609 │ 2.4810944589577892 │ 21.68 │ 6.937 │ -5.4 │ 16.6 │ 21.1 │ 26.4 │ 46.7 │ ▁▇▇▃ │ │
│ └────────────────┴────────┴─────────────────────┴───────┴───────┴───────┴──────┴──────┴──────┴──────┴────────┘ │
│ string │
│ ┏━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┓ │
│ ┃ ┃ ┃ ┃ ┃ ┃ ┃ ┃ chars per ┃ words ┃ total ┃ │
│ ┃ column ┃ NA ┃ NA % ┃ shortest ┃ longest ┃ min ┃ max ┃ row ┃ per row ┃ words ┃ │
│ ┡━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━┩ │
│ │ Date │ 0 │ 0 │ 2008-12- │ 2008-12- │ 2007-11- │ 2017-06- │ 10 │ 1 │ 145460 │ │
│ │ │ │ │ 01 │ 01 │ 01 │ 25 │ │ │ │ │
│ │ Location │ 0 │ 0 │ Sale │ Melbourn │ Adelaide │ Woomera │ 8.71 │ 1 │ 145460 │ │
│ │ │ │ │ │ eAirport │ │ │ │ │ │ │
│ │ WindGust │ 10326 │ 7.098858 │ W │ WNW │ E │ WSW │ 2.19 │ 0.93 │ 135134 │ │
│ │ Dir │ │ 79279527 │ │ │ │ │ │ │ │ │
│ │ WindDir9 │ 10566 │ 7.263852 │ W │ NNW │ E │ WSW │ 2.18 │ 0.93 │ 134894 │ │
│ │ am │ │ 60552729 │ │ │ │ │ │ │ │ │
│ │ │ │ 2 │ │ │ │ │ │ │ │ │
│ │ WindDir3 │ 4228 │ 2.906641 │ E │ WNW │ E │ WSW │ 2.21 │ 0.97 │ 141232 │ │
│ │ pm │ │ 00096246 │ │ │ │ │ │ │ │ │
│ │ │ │ 4 │ │ │ │ │ │ │ │ │
│ │ RainToda │ 3261 │ 2.241853 │ No │ Yes │ No │ Yes │ 2.22 │ 0.98 │ 142199 │ │
│ │ y │ │ 43049635 │ │ │ │ │ │ │ │ │
│ │ │ │ 6 │ │ │ │ │ │ │ │ │
│ │ RainTomo │ 3267 │ 2.245978 │ No │ Yes │ No │ Yes │ 2.22 │ 0.98 │ 142193 │ │
│ │ rrow │ │ 27581465 │ │ │ │ │ │ │ │ │
│ │ │ │ 7 │ │ │ │ │ │ │ │ │
│ └──────────┴───────┴──────────┴──────────┴──────────┴──────────┴──────────┴───────────┴──────────┴───────────┘ │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

## Exploring Variables (Columns) & Observations (Rows)

If we are going to narrow our focus to specific variables or groups of
observations, we need to know how to select columns, filter values, and
group the data. There are lots of different ways we can slice up the
data. We won’t cover all of them here[1], but we will try to cover a
range that helps illustrate how pandas works and will help you build the
intuition for working with data in pandas.

We can select columns in a variety of ways, but the “correct” way to
select columns in most circumstances is using selection brackets (the
square brackets `[]`), also known as the indexing operator.

[1] For more information, I’d recommend the [pandas
documentation](https://pandas.pydata.org/docs/), and this pandas
tutorial on [subsetting
data](https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html).

In [12]:
# selecting a single column by name
df['Date']

# alternative ways to select columns
# df.loc[:, 'Date']
# df.Date

0         2008-12-01
1         2008-12-02
2         2008-12-03
3         2008-12-04
4         2008-12-05
             ...    
145455    2017-06-21
145456    2017-06-22
145457    2017-06-23
145458    2017-06-24
145459    2017-06-25
Name: Date, Length: 145460, dtype: object

If we want to select multiple columns, we can use double squared
brackets (`[[ ]]`). This is the same process as before, but the inner
brackets define a list, and the outer are the selection brackets. Lists
are an example of a data (variable) type - for more detail on data
types, refer to [session 6](../06-data-types/index.qmd).

In [13]:
# selecting multiple columns (and all rows) by name
df[['Date', 'Location', 'Rainfall']]
# df.loc[:, ['Date', 'Location', 'Rainfall']]

145460 rows × 3 columns

While selection brackets are a quick and easy solution if we want to
grab a subset of variables in the dataset, it is realy only intended to
be used for simple operations using only column selection.

For row selection, we should use `pd.DataFrame.iloc[]`. The `iloc`
function is used for “integer position” selection, which means you can
select rows or columns using their integer position. For rows 10-15, you
can select them using the following:

In [14]:
# slicing by rows
df.iloc[10:16]

6 rows × 23 columns

We can do similar using a column’s integer position, but we have to
select all rows (`:`) first:

In [15]:
# using iloc with columns
df.iloc[:, 20]

0         21.8
1         24.3
2         23.2
3         26.5
4         29.7
          ... 
145455    22.4
145456    24.5
145457    26.1
145458    26.0
145459    20.9
Name: Temp3pm, Length: 145460, dtype: float64

Finally, we can put both together to take a subset of both rows and
columns:

In [16]:
# using iloc with rows and columns
df.iloc[10:16, 20]

10    28.8
11    17.0
12    15.8
13    19.8
14    23.5
15    26.2
Name: Temp3pm, dtype: float64

However, selecting by integer position is relatively limited. It is more
likely we would want to subset the data based on the values of certain
columns. We can filter rows by condition using `pd.DataFrame.loc[]`. The
`loc` function slices by label, instead of integer position.

For example, we might want to look at a subset of the data based on
location.

In [17]:
# select all observations in Perth
df.loc[df['Location'] == 'Perth'] # we use double equals to compare the value in Location to the string Perth 
                                  # (so the effect is to output the rows that match that condition.)

3193 rows × 23 columns

We can also filter by multiple values, such as location and rainfall.

In [18]:
df.loc[(df['Rainfall'] == 0) & (df['Location'] == 'Perth')]

2293 rows × 23 columns

For any complex process for subsetting the data, including multiple
conditions, `pd.DataFrame.loc[]` is the best bet.

### Summarising Data

Now that we know how to select the variables or observations we are
interested in, we can start doing some descriptive analysis. The
operations we use will depend on the questions we are trying to answer,
and the possibilities will be almost endless.

**Questions:**

-   What “functions” might we need to carry out on our data when we are
    exploring it?

We know that the weather data includes observations from all over the
country, but we might want to check exactly how many different locations
there are. We can use `pd.DataFrame.nunique()` to do this.

In [19]:
# count unique values
df['Location'].nunique()

49

We may also be interested in the locations themselves, which may tell us
more about the spatial distribution of our data. In this case, we can
use `pd.DataFrame.unique()`.

In [20]:
# get unique values
df['Location'].unique()

array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond',
       'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat',
       'Bendigo', 'Sale', 'MelbourneAirport', 'Melbourne', 'Mildura',
       'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns',
       'GoldCoast', 'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa',
       'Woomera', 'Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport',
       'Perth', 'SalmonGums', 'Walpole', 'Hobart', 'Launceston',
       'AliceSprings', 'Darwin', 'Katherine', 'Uluru'], dtype=object)

Another common operation we might look to do is calculating the mean
value (`pd.DataFrame.mean()`) of a certain variable. What is the average
value of sunshine across the entire dataset?

In [21]:
# calculate variable mean
df['Sunshine'].mean()

np.float64(7.6111775206611565)

This gives us the mean to many decimal places, and we probably don’t
need to know the average sunshine hours to this level of precision. We
can use the `pd.DataFrame.round()` function to round to two decimal
places.

In [22]:
# round mean value
df['Sunshine'].mean().round(2)

np.float64(7.61)

Many operations will return the value with information about the
object’s type included. The above values are wrapped in `np.float64()`
because `pd.DataFrame.mean()` uses numpy to calculate the mean value.
However, if you want to strip this information out so you only see the
value itself, you can use `print()`.

In [23]:
# print mean value
print(df['Sunshine'].mean().round(2))

7.61

While we are often interested in the mean value when we talk about
averages, we might want to know the median instead
(`pd.DataFrame.median()`).

In [24]:
# calculate other summary statistics
print(df['Sunshine'].median())

8.4

Another common calculation is summing values (`pd.DataFrame.sum()`). We
can use `sum()` to see the total hours of sunshine in our dataset, and
we can use `int()` to convert this value to an integer (which also means
we don’t need to use `print()`[1]).

[1] Some functions should be wrapped in `print()` in order to return a
value that is easy to read, but others won’t. There will be an internal
logic for which is which, but it’s not of huge importance to us. You are
better off just testing functions out and wrapping them in `print()` if
necessary.

In [25]:
# calculate sum value and return an integer
int(df['Sunshine'].sum())

575595

We can also apply these summary operations on multiple variables, using
the same selection logic as before (using double squared brackets).

In [26]:
print(df[['Sunshine', 'Rainfall']].mean())

Sunshine    7.611178
Rainfall    2.360918
dtype: float64

And we can apply multiple functions, using `pd.DataFrame.agg()`.

In [27]:
df['Sunshine'].agg(['mean', 'median', 'sum']).round(1)

mean           7.6
median         8.4
sum       575595.3
Name: Sunshine, dtype: float64

The next step when exploring specific variables will often be
group-level summaries. The average amount of sunshine across the whole
dataset has limited utility, but the average hours of sunshine in each
location allows us to compare between locations and start to understand
how different variables are related to each other. If we want to do a
group-level operation, we have to use `pd.DataFrame.groupby()`.

In [28]:
# calculate group means
df.groupby(by='Location')['Sunshine'].mean().round(1)

Location
Adelaide            7.7
Albany              6.7
Albury              NaN
AliceSprings        9.6
BadgerysCreek       NaN
Ballarat            NaN
Bendigo             NaN
Brisbane            8.1
Cairns              7.6
Canberra            7.4
Cobar               8.7
CoffsHarbour        7.4
Dartmoor            6.5
Darwin              8.5
GoldCoast           NaN
Hobart              6.6
Katherine           NaN
Launceston          NaN
Melbourne           6.4
MelbourneAirport    6.4
Mildura             8.5
Moree               8.9
MountGambier        6.5
MountGinini         NaN
Newcastle           NaN
Nhil                NaN
NorahHead           NaN
NorfolkIsland       7.0
Nuriootpa           7.7
PearceRAAF          8.8
Penrith             NaN
Perth               8.8
PerthAirport        8.8
Portland            6.5
Richmond            NaN
Sale                6.7
SalmonGums          NaN
Sydney              7.2
SydneyAirport       7.2
Townsville          8.5
Tuggeranong         NaN
Uluru  

The `groupby(by='Location')` function tells us the grouping variable
(location), then we select the variable we want to summarise by location
(sunshine), and then we specify the operation (mean).

There are multiple locations that return `NaN` (**N**ot **a**
**N**umber). This indicates that numpy was unable to calculate a mean
value for those locations. This is likely to be because all sunshine
values for those locations are null.

We can check this using `pd.DataFrame.count()`, which counts the total
non-null values (whereas `pd.DataFrame.size()` counts the total values).

In [29]:
# group by location and count non-null sunshine values
df.groupby('Location')['Sunshine'].count()

Location
Adelaide            1769
Albany              2520
Albury                 0
AliceSprings        2520
BadgerysCreek          0
Ballarat               0
Bendigo                0
Brisbane            3144
Cairns              2564
Canberra            1521
Cobar                550
CoffsHarbour        1494
Dartmoor            2566
Darwin              3189
GoldCoast              0
Hobart              3179
Katherine              0
Launceston             0
Melbourne           3192
MelbourneAirport    3008
Mildura             2876
Moree               2055
MountGambier        2597
MountGinini            0
Newcastle              0
Nhil                   0
NorahHead              0
NorfolkIsland       2570
Nuriootpa           2848
PearceRAAF          3004
Penrith                0
Perth               3188
PerthAirport        3004
Portland            2566
Richmond               0
Sale                1818
SalmonGums             0
Sydney              3328
SydneyAirport       2993
Townsville      

The results show that all the locations that return `NaN` in our group
mean calculation have zero non-null values.

## Transforming Data

Datasets are rarely perfectly clean and
[tidy](https://vita.had.co.nz/papers/tidy-data.pdf). We often need to
transform the data before we can get the most out of it.

**Questions:**

-   What sort of transformations would help us get the most out of the
    analysis of the Australian weather data?

The first step with any analysis is often converting columns to the
correct types. With a longitudinal (time-series) dataset,the date column
is a good place to start. We can use `pd.DataFrame.dtypes` to check the
data type, either of a single column (using the selector brackets) or
all columns in the dataset.

In [30]:
print(df.dtypes)

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RainTomorrow      object
dtype: object

All columns are either stored as `object` or `float64`. The `object`
data type is for generic non-numeric data, but from the columns that are
stored as objects, we can tell this is mostly categorical variables
where the categories are represented as text. The float64 data type
refers to data that is numeric and includes decimals (float64 = 64-bit
floating point number).

The date column is stored as an object, but pandas can store dates as
`datetime64`. We can convert dates using `pd.to_datetime()`. When
transforming data, if we want to keep those transformations, we have to
store those changes, using `=`. In this case, we want to convert the
date column but we don’t want to create an entirely new dataframe to
handle this change, so we can overwrite the current date column by using
the selection brackets to identify the column we want to apply this
change to.

In [31]:
# convert date column to datetime
df['Date'] = pd.to_datetime(df['Date'])

The remaining `object` columns can be converted to categorical, which
makes them easier to work with in subsequent analyses. We can use
`pd.DataFrame.astype()` to convert column data types.

In [32]:
# create a list of all object columns
object_cols = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']

# convert object columns to category
df[object_cols] = df[object_cols].astype('category')

A more efficient, though synactically more complex, way of doing this is
using lamda functions. We won’t cover lambda functons in this session
(they will be discussed in detail in a future session), but below is how
we can use them to convert objects to categories.

In [33]:
# convert object columns to category data type
df = df.apply(lambda x: x.astype('category') if x.dtype == 'object' else x)

Another choice we might make is to remove missing values, using
`pd.DataFrame.dropna()` to filter the null values and keep only the
non-null values. We can use this to drop all null values across the
entire dataset, or we can apply it to a subset of columns, using the
`subset` argument.

In [34]:
# filter observations where sunshine is NA
df.dropna(subset='Sunshine')

75625 rows × 23 columns

We haven’t stored this transformation, because filtering nulls without
careful consideration is a bad idea, but it’s useful to know,
nonetheless.

There are lots of ways we could transform the data, but the final
example we will consider here is reshaping the data using
`pd.DataFrame.pivot()`, which transforms the data from long to wide
format data, and `pd.DataFrame.melt()`, which transforms it from wide to
long format.

Perhaps we want to focus on the maximum temperature per day in each
location in 2015. We can use `pd.Series.dt.year` to get the year from
the date column, and filter for the year 2015, before reshaping the
data.

In [35]:
df2015 = df.loc[df['Date'].dt.year == 2015]
df_wide = df2015.pivot(index='Date', columns='Location', values='MaxTemp')

df_wide.head()

5 rows × 49 columns

Perhaps we want to look at the maximum and minimum temperatures in each
location, together. We can reshape the data to support this[1].

[1] This is often very useful when we need to visualise data, for
example plotting the max and min temp for each location, is easier if
the values are organised in the same column and differentiated using
another column.

In [36]:
df_long = df2015.melt(
    id_vars=['Date', 'Location'],
    value_vars=['MaxTemp', 'MinTemp'],
    var_name='Variable',
    value_name='Value'
)

df_long.head()

## Exercises

Some of these questions are easily answered by scrolling up and finding
the answer in the output of the above code, however, the goal is to find
the answer using code. No one actually cares what the answer to any of
these questions is, it’s the process that matters!

**Remember, if you don’t know the answer, it’s okay to Google it (or
speak to others, including me, for help)!**

In [37]:
# import the dataset
df = pd.read_csv('data/weatherAUS.csv')

1.  What is the ‘Sunshine’ column’s data type?

> **Solution**
>
> ``` python
> # What is the 'Sunshine' column's data type?
> print(df['Sunshine'].dtypes)
> ```
>
>     float64

1.  Identify all the columns that are of dtype ‘object’.

> **Solution**
>
> ``` python
> # Identify all the columns that are of dtype 'object'
> print(list(df.select_dtypes(include=['object'])))
> ```
>
>     ['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']

1.  How many of the dataframe’s columns are of dtype ‘object’?

> **Solution**
>
> ``` python
> # How many of the dataframe's columns are of dtype 'object'?
> len(list(df.select_dtypes(include=['object'])))
> ```
>
>     7

1.  How many of the ‘Rainfall’ column values are NAs?

> **Solution**
>
> ``` python
> # How many of the 'Rainfall' column values are NAs?
> print(df['Rainfall'].isna().sum())
> ```
>
>     3261

1.  Create a new dataframe which only includes the ‘Date’, ‘Location,
    ’Sunshine’, ‘Rainfall’, and ‘RainTomorrow’ columns.

> **Solution**
>
> ``` python
> new_df = df[['Date', 'Location', 'Sunshine', 'Rainfall', 'RainTomorrow']]
> new_df.head()
> ```
>
> <div>
> <style scoped>
>     .dataframe tbody tr th:only-of-type {
>         vertical-align: middle;
>     }
>
>     .dataframe tbody tr th {
>         vertical-align: top;
>     }
>
>     .dataframe thead th {
>         text-align: right;
>     }
> </style>
>
> |     | Date       | Location | Sunshine | Rainfall | RainTomorrow |
> |-----|------------|----------|----------|----------|--------------|
> | 0   | 2008-12-01 | Albury   | NaN      | 0.6      | No           |
> | 1   | 2008-12-02 | Albury   | NaN      | 0.0      | No           |
> | 2   | 2008-12-03 | Albury   | NaN      | 0.0      | No           |
> | 3   | 2008-12-04 | Albury   | NaN      | 0.0      | No           |
> | 4   | 2008-12-05 | Albury   | NaN      | 1.0      | No           |
>
> </div>

1.  Convert ‘RainTomorrow’ to a numeric variable, where ‘Yes’ = 1 and
    ‘No’ = 0.

> **Solution**
>
> ``` python
> # df['Location'].astype('category').cat.codes
> # df['RainTomorrow'].astype('category').cat.codes
> df['RainTomorrow'].map({'Yes': 1, 'No': 0})
> ```
>
>     0         0.0
>     1         0.0
>     2         0.0
>     3         0.0
>     4         0.0
>              ... 
>     145455    0.0
>     145456    0.0
>     145457    0.0
>     145458    0.0
>     145459    NaN
>     Name: RainTomorrow, Length: 145460, dtype: float64

1.  What is the average amount of rainfall for each location?

> **Solution**
>
> ``` python
> # average rainfall by location, sorted by value
> df.groupby('Location')['Rainfall'].mean().sort_values(ascending=False)
> ```
>
>     Location
>     Cairns              5.742035
>     Darwin              5.092452
>     CoffsHarbour        5.061497
>     GoldCoast           3.769396
>     Wollongong          3.594903
>     Williamtown         3.591108
>     Townsville          3.485592
>     NorahHead           3.387299
>     Sydney              3.324543
>     MountGinini         3.292260
>     Katherine           3.201090
>     Newcastle           3.183892
>     Brisbane            3.144891
>     NorfolkIsland       3.127665
>     SydneyAirport       3.009917
>     Walpole             2.906846
>     Witchcliffe         2.895664
>     Portland            2.530374
>     Albany              2.263859
>     BadgerysCreek       2.193101
>     Penrith             2.175304
>     Tuggeranong         2.164043
>     Dartmoor            2.146567
>     Richmond            2.138462
>     MountGambier        2.087562
>     Launceston          2.011988
>     Albury              1.914115
>     Perth               1.906295
>     Melbourne           1.870062
>     Watsonia            1.860820
>     PerthAirport        1.761648
>     Canberra            1.741720
>     Ballarat            1.740026
>     WaggaWagga          1.709946
>     PearceRAAF          1.669080
>     Moree               1.630203
>     Bendigo             1.619380
>     Hobart              1.601819
>     Adelaide            1.566354
>     Sale                1.510167
>     MelbourneAirport    1.451977
>     Nuriootpa           1.390343
>     Cobar               1.127309
>     SalmonGums          1.034382
>     Mildura             0.945062
>     Nhil                0.934863
>     AliceSprings        0.882850
>     Uluru               0.784363
>     Woomera             0.490405
>     Name: Rainfall, dtype: float64

1.  What is the average amount of rainfall for days that it will rain
    tomorrow?

> **Solution**
>
> ``` python
> # average rainfall depending on whether it will rain tomorrow or not
> df.groupby('RainTomorrow')['Rainfall'].mean()
> ```
>
>     RainTomorrow
>     No     1.270290
>     Yes    6.142104
>     Name: Rainfall, dtype: float64

1.  What is the average amount of sunshine in Perth when it will not
    rain tomorrow?

> **Solution**
>
> ``` python
> # average sunshine in Perth when it won't rain tomorrow
> df.loc[(df['Location'] == 'Perth') & (df['RainTomorrow'] == 'No'), 'Sunshine'].mean()
> # df[(df['Location']=='Perth') & (df['RainTomorrow']=='No')]['Sunshine'].mean()
> ```
>
>     np.float64(9.705306603773584)

1.  We want to understand the role that time plays in the dataset. Using
    the original dataframe, carry the following tasks and answer the
    corresponding questions:
    -   Create columns representing the year and month from the ‘Date’
        column. How many years of data are in the dataset?
    -   Examine the distribution of the ‘Sunshine’ NAs over time. Is
        time a component in the ‘Sunshine’ data quality issues?
    -   Calculate the average rainfall and sunshine by month. How do
        rainfall and sunshine vary through the year?
    -   Calculate the average rainfall and sunshine by year. How have
        rainfall and sunshine changed over time?

> **Solution**
>
> ``` python
> # get year and month columns
> df = (
>     df.assign(Date=pd.to_datetime(df['Date']))
>     .assign(
>         Year=lambda x: x['Date'].dt.year,
>         Month=lambda x: x['Date'].dt.month
>     )
> )
>
> # count unique years
> df['Year'].nunique()
> ```
>
>     11
>
> ``` python
> # lambda function counting nulls by year
> df.groupby('Year')['Sunshine'].apply(lambda x: x.isna().sum())
> ```
>
>     Year
>     2007        0
>     2008      323
>     2009     6146
>     2010     6220
>     2011     6053
>     2012     6539
>     2013     7570
>     2014     9157
>     2015     9441
>     2016    11994
>     2017     6392
>     Name: Sunshine, dtype: int64
>
> ``` python
> # rainfall and sunshine by month
> df.groupby('Month')[['Rainfall', 'Sunshine']].mean().round(1)
> ```
>
> <div>
> <style scoped>
>     .dataframe tbody tr th:only-of-type {
>         vertical-align: middle;
>     }
>
>     .dataframe tbody tr th {
>         vertical-align: top;
>     }
>
>     .dataframe thead th {
>         text-align: right;
>     }
> </style>
>
> |       | Rainfall | Sunshine |
> |-------|----------|----------|
> | Month |          |          |
> | 1     | 2.7      | 9.2      |
> | 2     | 3.2      | 8.6      |
> | 3     | 2.8      | 7.6      |
> | 4     | 2.3      | 7.1      |
> | 5     | 2.0      | 6.3      |
> | 6     | 2.8      | 5.6      |
> | 7     | 2.2      | 6.1      |
> | 8     | 2.0      | 7.1      |
> | 9     | 1.9      | 7.7      |
> | 10    | 1.6      | 8.5      |
> | 11    | 2.3      | 8.7      |
> | 12    | 2.5      | 9.0      |
>
> </div>
>
> ``` python
> # rainfall and sunshine by year
> df.groupby('Year')[['Rainfall', 'Sunshine']].mean().round(1)
> ```
>
> <div>
> <style scoped>
>     .dataframe tbody tr th:only-of-type {
>         vertical-align: middle;
>     }
>
>     .dataframe tbody tr th {
>         vertical-align: top;
>     }
>
>     .dataframe thead th {
>         text-align: right;
>     }
> </style>
>
> |      | Rainfall | Sunshine |
> |------|----------|----------|
> | Year |          |          |
> | 2007 | 3.2      | 8.1      |
> | 2008 | 2.3      | 7.8      |
> | 2009 | 2.2      | 7.9      |
> | 2010 | 2.7      | 7.3      |
> | 2011 | 2.8      | 7.3      |
> | 2012 | 2.4      | 7.6      |
> | 2013 | 2.3      | 7.7      |
> | 2014 | 2.0      | 7.8      |
> | 2015 | 2.2      | 7.7      |
> | 2016 | 2.4      | 7.6      |
> | 2017 | 2.5      | 7.7      |
>
> </div>